# runtime measure

In [1]:
import time
import ctypes

import tvm
from tvm import te
from tvm.contrib.utils import tempdir
from tvm.runtime.module import BenchmarkResult

In [2]:
tmp = tempdir()
filename = tmp.relpath("log")

@tvm.register_func
def my_debug(filename):
    """one call lasts for 100 ms and writes one character to a file"""
    time.sleep(0.1)
    with open(filename, "a") as fout:
        fout.write("c")

X = te.compute((), lambda: tvm.tir.call_packed("my_debug", filename))
s = te.create_schedule(X.op)
func = tvm.build(s, [X])

x = tvm.nd.empty((), dtype="int32")
ftimer = func.time_evaluator(func.entry_name, tvm.cpu(), number=1, repeat=1)
ftimer(x)

with open(filename, "r") as fin:
    ct = len(fin.readline())

assert ct == 2

ftimer = func.time_evaluator(func.entry_name, tvm.cpu(), number=1, repeat=1, min_repeat_ms=1000)
ftimer(x)

# make sure we get more than 10 calls
with open(filename, "r") as fin:
    ct = len(fin.readline())

assert ct > 10 + 2